In [ ]:
import os
import awswrangler as wr
from dotenv import load_dotenv
from s3pathlib import S3Path

if "notebook" in os.getcwd():
    os.chdir("..")

from src.notebook_util import setup_epa_s3_session

In [ ]:
epa_boto3_session = setup_epa_s3_session()

bird_obj_list = wr.s3.list_objects(
    f"s3://{os.environ['epa_bucket']}/ARU/2022/Cuyahoga River/AVCR-6/Card 2 0614_0802/Acoustic/",
    boto3_session=epa_boto3_session
)
bird_obj_list

In [ ]:
def eval_object_length(s3uri, target_size, boto_session):
    result = wr.s3.describe_objects(
        s3uri,
        boto3_session=boto_session,
    )

    object_size = int(result[s3uri]["ContentLength"])

    if object_size > target_size:
        return True
    else:
        return False

In [ ]:
counter = 0

for s3_file_path in bird_obj_list:
    epa_s3path = S3Path.from_s3_uri(s3_file_path)

    epa_s3path_parts = epa_s3path.parts
    epa_s3path_parent = epa_s3path.parent

    mtsu_s3path = epa_s3path.copy()
    mtsu_s3path = mtsu_s3path.change(new_bucket="mtsu-bird-sound-data-bucket")
    mtsu_s3path_parent = mtsu_s3path.parent

    if "Acoustic" in epa_s3path_parts:
        if eval_object_length(epa_s3path.uri, target_size=50000000, boto_session=epa_boto3_session):

            print(epa_s3path.uri)
            print(epa_s3path_parent.uri)
            print(mtsu_s3path_parent.uri)
            
        wr.s3.copy_objects(
            paths=[epa_s3path.uri],
            source_path=epa_s3path_parent.uri,
            target_path=mtsu_s3path_parent.uri,
            boto3_session=epa_boto3_session,
        )

        counter += 1

        if counter > 20:
            break